<a href="https://colab.research.google.com/github/dtabuena/CrispyMice/blob/main/CrispyWrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def crispy_mouse_wrap(dataset,VC_prot,IC_prot):

    '''Unpack'''
    data_name = dataset['data_name']
    data_source = dataset['data_source']
    file_naming_scheme = dataset['file_naming_scheme']


    '''Set Internal Analysis Params'''
    spike_args =  {'spike_thresh':20, 'high_dv_thresh': 25,'low_dv_thresh': -5,'window_ms': 2}

    func_dict = {}
    arg_dict = {}

    func_dict['VC - 3min GapFree']= rmp_analyzer
    arg_dict['VC - 3min GapFree'] = [False]

    func_dict['IC - Rheobase']= rheobase_analyzer
    arg_dict['IC - Rheobase'] = [spike_args, False, False, False]  # [spike_args, to_plot, verbose, force_singlespike]

    func_dict['IC - Gain - D20pA']= gain_analyzer
    arg_dict['IC - Gain - D20pA']= [spike_args, 0.8, 0]  # [spike_args, to_plot, verbose, force_singlespike]
    func_dict['IC - Gain - D50pA']= func_dict['IC - Gain - D20pA'] 
    arg_dict['IC - Gain - D50pA']= arg_dict['IC - Gain - D20pA']

    func_dict['VC - MemTest-10ms-160ms']= membrane_analyzer
    arg_dict['VC - MemTest-10ms-160ms']= [False, False, ['Ra', 'Rm', 'Cm', 'tau',	'Cmq',	'Cmf',	'Cmqf', 'Cm_pc']]  # [to_plot, verbose]

    func_dict['IC - Latentcy 800pA-1s']= latencey_analyzer 
    arg_dict['IC - Latentcy 800pA-1s']= [spike_args, False]  # [spike_args, to_plot]

    func_dict['IC - Latentcy 800pA-1s']= latencey_analyzer 
    arg_dict['IC - Latentcy 800pA-1s']= [spike_args, False]  # [spike_args, to_plot]

    func_dict['IC - R input']= input_resistance_analyzer 
    arg_dict['IC - R input']= [[-30, 10] ,False]  # [dVm_limits, to_plot]

    func_dict['VC - Multi IV - 150ms'] = IV_analyzer
    arg_dict['VC - Multi IV - 150ms']= [{'IV_Early':(16.5, 30),'IV_Steady_State':(100,120)} ,False]  # [measure_windows, to_plot]



    abf_recordings_df,protocol_set = catalogue_recs(file_loc,file_naming_scheme)

    cell_prot_lut(abf_recordings_df,protocol_set,csv_name=data_name+'_LUT')


    file_loc = get_drobox_folder(data_source, 'my_ephys_data.zip')
    clear_output(wait=False)    

    abf_recordings_df, problem_recs = analysis_iterator(abf_recordings_df,func_dict,arg_dict)
    print(problem_recs) 

    cell_df = cell_sorting(abf_recordings_df)

    '''Consolidate to Cells'''
    list_types = ['Recording_name','protocol','abf_timestamp', 'channelList']
    any_types = ['Rec_date',	'Virus',	'GenoType',	'Sex',	'Age',	'Slice_Num',  'Cell_num', 'Cell_Type']
    cell_df_con = cell_consolidation(cell_df,list_types,any_types)
    cols_to_simplify = ['IV_Early', 'IV_Steady_State']
    cell_df_nd = simplify_dicts(cell_df_con,cols_to_simplify)     

    '''Make Excell Friendly'''
    keys_and_data_cols={'Stim_Levels_(pA)': ['Stim_Levels_(pA)', 'Spike_Counts' ],
                    'IV_Early_(V_stim)': ['IV_Early_(V_stim)', 'IV_Early_(I_peak)', 'IV_Steady_State_(I_mean)']}
    cell_df_csv = csv_frinedly(cell_df_nd,keys_and_data_cols)
    
    ''' Convert to Current Density'''
    size_col = 'Cmq_160.0'
    current_col_list = ['IV_Early_(I_peak)_', 'IV_Steady_State_(I_mean)_']
    cell_df_csv = current_density_correction(cell_df_csv, size_col, current_col_list)


    '''Abridge DataFrame'''
    abrg_exclusions = ['Recording_name', 
                    'protocol', 'abf_timestamp', 'channelList',  'Ra_10.0', 'Rm_10.0', 'tau_10.0', 'Cmq_10.0', 'Cmf_10.0',
                    'Cmqf_10.0',  'Cmf_160.0', 'Cmqf_160.0', 'Cm_pc_160.0',
                    'Gain_R2', 'Stim_Levels_(pA)', 'Spike_Counts', 'Firing_Duration_%', 'Gain_Vh',  'Vhold_spike',
                     'Rin_Rsqr',  'Ramp_AP_thresh', 'Ramp_Vh', 'Ramp_Rheobase', 
                    'ap_thresh_us', 'v_half',
                    'IV_Early_(range)', 'IV_Early_(I_peak)', 'IV_Early_(I_mean)', 'IV_Early_(V_stim)', 'IV_Steady_State_(range)',
                    'IV_Steady_State_(I_peak)', 'IV_Steady_State_(I_mean)', 'IV_Steady_State_(V_stim)', ]

    abrg_keep = [c for c in cell_df_csv.columns if c not in abrg_exclusions]
    cell_df_csv_abrg = cell_df_csv[abrg_keep]

    '''Stratify Cells By Type'''
    strat_col = 'Cell_Type'
    xl_file_name= data_name+'.xlsx'
    new_dfs = stratify_cells(cell_df_csv_abrg,strat_col,xl_file_name)

    return new_dfs, cell_df_csv_abrg, cell_df_csv, cell_df_nd, cell_df, abf_recordings_df



In [ ]:
dataset = {'data_name': 'RNF182',
           'data_source': "https://www.dropbox.com/sh/n9t8p257wnzlijk/AAC9Z36JodisyZjnM3mkJC3Xa?dl=0",
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }

VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree']

crispy_mouse_wrap(dataset,VC_prot,IC_prot)